# Тестовое задание Kazan Express

## Задача

В маркетплейс каждый день поступает множество новых товаров и каждый из них необходимо отнести в определенную категорию в нашем дереве категорий. На это тратится много сил и времени, поэтому необходиимо научиться предсказывать категорию на основе названий и параметров товаров.

## Входные данные
- categories_tree.csv - файл с деревом категорий на маркетплейсе
- train.parquet - файл с товарами на маркетплейсе
- test.parquet - файл идентичный train.parquet, но без реального category_id

## Пути решения

В ходе исследования появилось несколько путей решения: 
1. Предсказать конечную листовую категорию с помощью OneVSRest
2. Предсказать весь путь по категориям с помощью MultiOutputClassifier
3. Предсказать весь путь по категориям, исходя из иерархичной структуры категорий

Также было несколько вариантов работы с текстом - TfidfVectorizer и BERT.

### OneVSRest
Если предсказывать исключительно конечную листовую категорию, то можно свести задачу к мультиклассовой. В таком случае классификация может быть "один против всех", "каждый против каждого". В ходе работы удалось реализовать метод OneVSRest с использованием метода опорных векторов LinearSVC. Значения в результате получились следующие Precision = 0.83, recall = 0.83,  F1-score = 0.82

### MultiOutputClassifier
Для предсказания всего пути был выбран метод MultiOutputClassifier с использованием метода опорных векторов LinearSVC. Задачу предсказания всего пути можно свести к multi label classification. Чтобы модель выдавала на выходе сразу весь путь необходимо бинамизировать входной таргет, что и было сделано с помощью MultiLabelBinarizer. Всего имеется 1475 категорий на разных уровнях. Каждому объекту может соответствовать некоторое количество этих категорий. На выходе модель как раз их и выдает. Значения в результате получились следующие Precision = 0.9, recall = 0.85,  F1-score = 0.87. Модель выводит результаты в виде массивов, которые в дальнейшем можно сопоставить с категориям. 

### Иерархичная структура категорий
Была попытка пойти по-другому пути решения задачи и воспользоваться HierarchicalClassifier, который принимает на вход в том числе словарь с категориям. Однако, процесс обучения занял бы длительное время, также появлялись ошибки. В пункте 8.  Addition hierarchical отражен код. 

### TfidfVectorizer
Модель принимает на вход преобразованный признак "title". Обработка текста: в тексте были оставлены только русские буквы, убраны слова менее 4 букв (так как после предыдущего шага оставались обозначения, например, "мл", "шт" и тд), произведена лемматизация и убраны стопслова. В качестве векторайзера в итоге используется TfidfVectorizer.

### BERT
BERT реализовывался в Google Colab с использованием GPU. Изначально планировалось обучать модель на полученном наборе эмбэддингов после BERT. Однако, вероятнее всего, не хватало мощности ноутбука и мощностей Google Colab, чтобы модель полностью обучилась. В пункте 9. Addition BERT добавлен код из Google Colab, в результате которого получились эмбэддинги. Также в папку solution добавлен файл с эмбэддингами features_kazan.csv. 

# Imports

In [2]:
import pandas as pd
import numpy as np
! pip install  transformers fast_ml pyarrow
from fast_ml.model_development import train_valid_test_split
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from tqdm.auto import tqdm

import transformers
import re 
from tqdm import notebook
from pymystem3 import Mystem
m = Mystem()
! pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

! pip install sklearn_hierarchical_classification
from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
from sklearn.metrics import classification_report
from sklearn import svm

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
categories = pd.read_csv('/Users/kate/Desktop/DS/yandex/KazanEpress/categories_tree.csv')

In [4]:
categories_full = pd.read_csv('/Users/kate/Desktop/DS/yandex/KazanEpress/categories_tree.csv')

In [5]:
data = pd.read_parquet('/Users/kate/Desktop/DS/yandex/KazanEpress/train.parquet', engine='pyarrow')

In [6]:
categories = categories.drop(['title'], axis = 1)

In [7]:
categories

,id,parent_id
0,1,0
1,114,1913
2,115,328
3,128,2475
4,131,2475
...,...,...
3365,14555,11691
3366,14556,10062
3367,14557,2894
3368,14558,10092


In [8]:
new_data = data.merge(categories, left_on = 'category_id', right_on = 'id')

In [9]:
new_data.rename(columns = {'id_x':'number'}, inplace = True)

In [10]:
new_data

,number,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id,id_y,parent_id
0,1267423,Muhle Manikure Песочные колпачки для педикюра ...,Muhle Manikure Колпачок песочный шлифовальный ...,None,0.000000,0,2693,2693,10355
1,93588,"Маникюрное копытце для обработки ногтей, пушер",None,None,4.774194,31,2693,2693,10355
2,1293082,Апельсиновые палочки для маникюра и педикюра 1...,Апельсиновые палочки для маникюра и педикюра 1...,None,5.000000,1,2693,2693,10355
3,321741,Щеточка для маникюра круглая,None,Вид:Круглая,5.000000,3,2693,2693,10355
4,1447598,Тонкие деревянные ватные палочки микробраши,None,None,0.000000,0,2693,2693,10355
...,...,...,...,...,...,...,...,...,...
283447,1303381,Купальник 3 цвета PLUS SIZE/размер плюс/больши...,"Удобный купальник для бассейна и пляжа, размер...",российский размер:50|52|54|56,5.000000,2,2832,2832,2812
283448,1480283,"Комбинезон мужской флисовый ""Ironcust""",None,None,0.000000,0,2603,2603,10445
283449,1480262,"Комбинезон мужской флисовый ""Ironcust""",None,None,0.000000,0,2603,2603,10445
283450,1480290,"Комбинезон мужской Ironcust, флисовый",None,None,0.000000,0,2603,2603,10445


# Working with text

In [11]:
def clear_text(text):
    new_text = []
    for i in text:
        new_text.append((re.sub(r'[^а-яА-ЯёЁ]', ' ', str(i).lower()).split()))
    return new_text

def string(text):
  new_text = []
  for i in text:
    line = []
    line = [x for x in i if len(x)>3]
    
    new_text.append(line)
  return new_text

def lemmatize2(text):
    
    new_text = []

    for i in text:
      words = i # разбиваем текст на слова    
      res = []

      for word in words:
          p = morph.parse(word)[0]
          res.append(p.normal_form)
      new_text.append(res)

    return new_text

nltk.download('stopwords')

stop_words_rus = set(nltk_stopwords.words('russian'))

def stopwords(text):
    new_text = []
    for word_list in text:
        line = []
        line = [x for x in word_list if not x in stop_words_rus]

        new_text.append(' '.join(line))
    return new_text

[nltk_data] Downloading package stopwords to /Users/kate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
new_data['title'] = stopwords(lemmatize2(string(clear_text(new_data['title']))))
#new_data['short_description'] = stopwords(lemmatize2(string(clear_text(new_data['short_description']))))
new_data[:10]

,number,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id,id_y,parent_id
0,1267423,песочный колпачок педикюр средний грита упаковка,Muhle Manikure Колпачок песочный шлифовальный ...,None,0.000000,0,2693,2693,10355
1,93588,маникюрный копытце обработка ноготь пушер,None,None,4.774194,31,2693,2693,10355
2,1293082,апельсиновый палочка маникюр педикюр штука,Апельсиновые палочки для маникюра и педикюра 1...,None,5.000000,1,2693,2693,10355
3,321741,щёточка маникюр круглый,None,Вид:Круглая,5.000000,3,2693,2693,10355
4,1447598,тонкий деревянный ватный палочка микробраша,None,None,0.000000,0,2693,2693,10355
5,1621957,наклейка типёс развод лист штука,None,None,0.000000,0,2693,2693,10355
6,807873,апельсиновый палочка маникюр педикюр упаковка ...,None,None,5.000000,3,2693,2693,10355
7,1150961,апельсиновый палочка маникюр,Незаменимый инструмент для маникюра и педикюра...,None,0.000000,0,2693,2693,10355
8,1267598,песочный колпачок педикюр супер грубый грита у...,Muhle Manikure Песочные колпачки для педикюра ...,None,0.000000,0,2693,2693,10355
9,891758,маникюрный подставка прямой ножка,Подставка для маникюра,None,0.000000,0,2693,2693,10355


# ML (prediction of the ultimate category)

In [253]:
X_train, X_valid, y_train, y_valid = train_test_split(new_data['title'], 
                                                      new_data['category_id'], train_size=0.9, random_state = 42)

In [254]:
corpus = X_train.values.astype('U')
corpus2 = X_valid.values.astype('U')

In [255]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(corpus)
_vectorizer = vectorizer
X_valid = _vectorizer.transform(corpus2)

In [25]:
model = OneVsRestClassifier(DecisionTreeClassifier(random_state=42)).fit(X_train, y_train)
prediction = model.predict(X_valid)
print(classification_report(y_valid, prediction))

              precision    recall  f1-score   support

        2598       0.00      0.00      0.00         1
        2599       0.95      0.94      0.95       109
        2600       1.00      0.64      0.78        14
        2601       0.64      0.22      0.33        96
        2602       0.79      0.68      0.73        38
        2604       0.00      0.00      0.00         3
        2605       0.80      0.55      0.65        29
        2607       0.96      0.71      0.82       145
        2608       0.50      0.45      0.47        20
        2610       0.83      0.87      0.85       197
        2631       0.94      0.81      0.87       125
        2632       0.98      0.90      0.93        48
        2633       0.75      0.50      0.60        12
        2634       0.72      0.67      0.70       190
        2635       0.88      0.48      0.62        46
        2636       0.79      0.47      0.59        70
        2662       0.85      0.70      0.77        74
        2663       0.88    

In [256]:
model = OneVsRestClassifier(LinearSVC(random_state=42)).fit(X_train, y_train)
prediction = model.predict(X_valid)

In [257]:
from sklearn.metrics import classification_report
print(classification_report(y_valid, prediction))

              precision    recall  f1-score   support

        2599       0.95      0.97      0.96        39
        2600       0.83      1.00      0.91         5
        2601       0.59      0.24      0.34        41
        2602       0.95      0.91      0.93        22
        2604       0.00      0.00      0.00         0
        2605       1.00      0.70      0.82        10
        2607       0.86      0.75      0.80        56
        2608       0.43      0.38      0.40         8
        2610       0.82      0.91      0.86        78
        2631       0.96      0.90      0.93        60
        2632       0.80      1.00      0.89        16
        2633       1.00      0.25      0.40         4
        2634       0.71      0.76      0.73        91
        2635       0.80      0.50      0.62        16
        2636       0.84      0.80      0.82        46
        2662       0.88      0.85      0.87        27
        2663       0.93      0.91      0.92        56
        2672       1.00    

In [258]:
print('Precision  recall  F1-score', 
      precision_recall_fscore_support(y_valid,prediction, average='weighted'))

Precision  recall  F1-score (0.8267334228511989, 0.829252804628519, 0.8193511193698777, None)


In [214]:
test_data = pd.read_parquet('/Users/kate/Desktop/DS/yandex/KazanEpress/test.parquet', engine='pyarrow')

In [215]:
test_data

,id,title,short_description,name_value_characteristics,rating,feedback_quantity
0,1070974,Браслет из натуральных камней LOTUS,None,None,0.000000,0
1,450413,Fusion Life - Шампунь для сухих и окрашенных в...,None,None,4.333333,6
2,126857,"Микрофон для ПК jack 3,5мм всенаправленный","универсальный 3,5 мм микрофон запишет ваш звук",None,3.708333,24
3,1577569,Серьги гвоздики сердце,Серьги гвоздики сердце,None,0.000000,0
4,869328,"Чёрно-красная стильная брошь ""Тюльпаны"" из акр...",Стильная и яркая брошь ручной работы! Великоле...,None,0.000000,0
...,...,...,...,...,...,...
70859,967535,Носки с мехом куницы авокадо разноцветные,Пуховые носки с мехомом куницы с авакадо.,None,5.000000,3
70860,1488636,"Эфирное масло Сосны, 10 мл, от КедрМаркет","Масло сосны повышает защитную функцию кожи, уп...",None,0.000000,0
70861,827510,Компект (футболка+шорты),"Отличный комплект. Удобный, комфортный.",None,0.000000,0
70862,529244,Купальный костюм Mark Formelle,None,Российский размер:40|42|44|46,0.000000,0


-----------


# Full path of categories

In [13]:
new_data = new_data.drop(['id_y'], axis = 1)

In [14]:
new_data = new_data.merge(categories, left_on = 'parent_id', right_on = 'id', how = 'left')

In [15]:
new_data = new_data.drop(['parent_id_x'], axis = 1)

In [16]:
new_data = new_data.merge(categories, left_on = 'parent_id_y', right_on = 'id', how = 'left')

In [17]:
new_data = new_data.drop(['parent_id_y'], axis = 1)

In [18]:
new_data.rename(columns = {'id_x':'parent_1', 'id_y':'parent_2', 'parent_id':'parent_3'}, inplace = True)

In [19]:
new_data = new_data.merge(categories, left_on = 'parent_3', right_on = 'id', how = 'left')

In [20]:
new_data = new_data.merge(categories, left_on = 'parent_id', right_on = 'id', how = 'left')

In [21]:
new_data = new_data.drop(['id_x', 'parent_id_x'], axis = 1)

In [22]:
new_data = new_data.merge(categories, left_on = 'parent_id_y', right_on = 'id', how = 'left')

In [23]:
new_data = new_data.drop(['id'], axis = 1)

In [24]:
new_data.rename(columns = {'id_y':'parent_4', 'parent_id_y':'parent_5', 'parent_id':'parent_6'}, inplace = True)

In [25]:
new_data

,number,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id,parent_1,parent_2,parent_3,parent_4,parent_5,parent_6
0,1267423,песочный колпачок педикюр средний грита упаковка,Muhle Manikure Колпачок песочный шлифовальный ...,None,0.000000,0,2693,10355,10113,10012,1.0,0.0,NaN
1,93588,маникюрный копытце обработка ноготь пушер,None,None,4.774194,31,2693,10355,10113,10012,1.0,0.0,NaN
2,1293082,апельсиновый палочка маникюр педикюр штука,Апельсиновые палочки для маникюра и педикюра 1...,None,5.000000,1,2693,10355,10113,10012,1.0,0.0,NaN
3,321741,щёточка маникюр круглый,None,Вид:Круглая,5.000000,3,2693,10355,10113,10012,1.0,0.0,NaN
4,1447598,тонкий деревянный ватный палочка микробраша,None,None,0.000000,0,2693,10355,10113,10012,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283447,1303381,купальник цвет размер плюс больший размер,"Удобный купальник для бассейна и пляжа, размер...",российский размер:50|52|54|56,5.000000,2,2832,2812,2808,2807,10014.0,1.0,0.0
283448,1480283,комбинезон мужской флисовыя,None,None,0.000000,0,2603,10445,10052,10014,1.0,0.0,NaN
283449,1480262,комбинезон мужской флисовыя,None,None,0.000000,0,2603,10445,10052,10014,1.0,0.0,NaN
283450,1480290,комбинезон мужской флисовыя,None,None,0.000000,0,2603,10445,10052,10014,1.0,0.0,NaN


In [26]:
columns = ['category_id', 'parent_1', 'parent_2', 'parent_3', 'parent_4', 'parent_5', 'parent_6']

mas = []
for i in range(len(new_data)):
    path = []
    for column in columns:

        if new_data[column][i] > 1:
            path.append(new_data[column][i])
        else:
            continue
    mas.append(path)

In [27]:
new_data['path1'] = mas

In [28]:
new_data

,number,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id,parent_1,parent_2,parent_3,parent_4,parent_5,parent_6,path1
0,1267423,песочный колпачок педикюр средний грита упаковка,Muhle Manikure Колпачок песочный шлифовальный ...,None,0.000000,0,2693,10355,10113,10012,1.0,0.0,NaN,"[2693, 10355, 10113, 10012]"
1,93588,маникюрный копытце обработка ноготь пушер,None,None,4.774194,31,2693,10355,10113,10012,1.0,0.0,NaN,"[2693, 10355, 10113, 10012]"
2,1293082,апельсиновый палочка маникюр педикюр штука,Апельсиновые палочки для маникюра и педикюра 1...,None,5.000000,1,2693,10355,10113,10012,1.0,0.0,NaN,"[2693, 10355, 10113, 10012]"
3,321741,щёточка маникюр круглый,None,Вид:Круглая,5.000000,3,2693,10355,10113,10012,1.0,0.0,NaN,"[2693, 10355, 10113, 10012]"
4,1447598,тонкий деревянный ватный палочка микробраша,None,None,0.000000,0,2693,10355,10113,10012,1.0,0.0,NaN,"[2693, 10355, 10113, 10012]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283447,1303381,купальник цвет размер плюс больший размер,"Удобный купальник для бассейна и пляжа, размер...",российский размер:50|52|54|56,5.000000,2,2832,2812,2808,2807,10014.0,1.0,0.0,"[2832, 2812, 2808, 2807, 10014.0]"
283448,1480283,комбинезон мужской флисовыя,None,None,0.000000,0,2603,10445,10052,10014,1.0,0.0,NaN,"[2603, 10445, 10052, 10014]"
283449,1480262,комбинезон мужской флисовыя,None,None,0.000000,0,2603,10445,10052,10014,1.0,0.0,NaN,"[2603, 10445, 10052, 10014]"
283450,1480290,комбинезон мужской флисовыя,None,None,0.000000,0,2603,10445,10052,10014,1.0,0.0,NaN,"[2603, 10445, 10052, 10014]"


# MultiLabelBinarizer

In [30]:
mlb = MultiLabelBinarizer()
label = mlb.fit_transform(new_data['path1'])

In [31]:
mlb.classes_

array([2598, 2599, 2600, ..., 14553, 14557, 14559], dtype=object)

In [32]:
label

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [33]:
new_data = new_data.join(pd.DataFrame(mlb.fit_transform(new_data.pop('path1')), index=new_data.index, columns=mlb.classes_))

In [34]:
for column in new_data[new_data.columns[13:]]:
    if new_data[column].unique().tolist() == [0, 1] or new_data[column].unique().tolist() == [1, 0]:
        continue
    else:
        print(column)

# ML (prediction of the full path)

In [35]:
X_train, X_valid, y_train, y_valid = train_test_split(new_data['title'], 
                                                      new_data[new_data.columns[13:]], train_size=0.90, random_state = 42)

In [36]:
y_train.shape

(255106, 1475)

In [37]:
corpus = X_train.values.astype('U')
corpus2 = X_valid.values.astype('U')

In [38]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(corpus)
_vectorizer = vectorizer
X_valid = _vectorizer.transform(corpus2)

In [39]:
clf = MultiOutputClassifier(LinearSVC(random_state=42))
clf.fit(X_train, y_train)
clf_predictions = clf.predict(X_valid)


print('Precision   recall  F1 score', 
      precision_recall_fscore_support(y_valid,clf_predictions, average='weighted'))

Precision   recall  F1 score (0.9077331941253716, 0.8464289929278899, 0.8683247434255759, None)


In [41]:
pred = mlb.inverse_transform(clf.predict(X_valid[11]))
for i in pred:
    pred = (list(i))
    print((pd.DataFrame(pred)
     .merge(categories_full, left_on = 0, right_on = 'id', how = 'left')
     .drop('id', axis = 1)
     .rename(columns = {0:'pred'})))

    pred                   title  parent_id
0  10014                  Одежда          1
1  10116          Женская одежда      10014
2  10533  Колготки, носки, чулки      10116
3  13143     Носки и подследники      10533


# Addition hierarchical

In [232]:
# categories = categories[['parent_id', 'id']].values.tolist()

получение словаря с категориями

In [233]:
# from collections import defaultdict
# d = defaultdict(list)
# for k, v in categories:
#    d[k].append(v)
# d.items()

dict_items([(0, [1, 553, 1754, 2000]), (1913, [114, 329, 366, 14350, 14351, 14352, 14353, 14354, 14355, 14356, 14357]), (328, [115, 371, 14318, 14319, 14320, 14321]), (2475, [128, 131, 132, 133, 135, 136, 138, 142, 531, 2477, 14287, 14288, 14289, 14290, 14291, 14292, 14293, 14294, 14363]), (14283, [134, 14325, 14326, 14327]), (2478, [143, 555, 14295, 14296, 14297, 14298, 14299, 14300, 14301, 14302, 14303, 14304, 14305]), (2481, [326, 14322, 14323, 14324]), (1821, [328, 1913, 2475, 2476, 2478, 2480, 2481, 2483, 14283, 14285]), (2483, [356, 369, 1892, 1915, 14309, 14310, 14311, 14312, 14313, 14314, 14315, 14316, 14317]), (1, [1821, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10010, 10011, 10012, 10013, 10014, 10015, 10016, 10018, 10019, 10020]), (10916, [2570, 12029, 12800, 13489, 13958, 13993]), (11454, [2571, 2572, 2573, 2574, 11992, 12089, 13191, 13233, 13351, 13374, 13398, 14068, 14257]), (10192, [2575, 12151, 12187, 12702, 13294]), (10216, [2576, 2577, 11556, 12037, 1277

In [234]:
# from sklearn_hierarchical_classification.constants import ROOT
# d[ROOT] = d.pop(0)

In [235]:
# ! pip install sklearn_hierarchical_classification
# from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
# from sklearn.metrics import classification_report
# from sklearn import svm

In [ ]:
# base_estimator = svm.SVC()
# class_hierarchy = d
# clf = HierarchicalClassifier(
#     base_estimator=base_estimator,
#     class_hierarchy=class_hierarchy,
#     progress_wrapper=notebook.tqdm,
# )
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_valid)

# print("Classification Report:\n", classification_report(y_valid, y_pred))

# Addition BERT

In [ ]:
# tokenizer = transformers.AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

In [ ]:
# encoded_title = tokenizer(list(new_data['title']), add_special_tokens=True, padding = True, 
#                          return_attention_mask = True, return_tensors = "pt")
# encoded_title

In [ ]:
# model = transformers.AutoModel.from_pretrained("cointegrated/rubert-tiny")
# model.cuda()
# encoded_title = {k: v.cuda() for k, v in encoded_title.items()}
# encoded_desc = {k: v.cuda() for k, v in encoded_desc.items()}

In [ ]:
# def embedding(text):
#     batch_size = 128
#     embeddings = []
#     with torch.no_grad():
#         for i in notebook.tqdm(range(0, len(text['input_ids']), batch_size)):
#             bert_embeddings = model(
#                 input_ids = text['input_ids'][i: i + batch_size], 
#                 attention_mask = text['attention_mask'][i: i + batch_size]
#             )
#             embeddings.append(bert_embeddings[0][:,0,:].cpu().numpy()) 
#             # embeddings.append(bert_embeddings[0][:,0,:].numpy()) 
#     return embeddings

In [ ]:
# embeddings_title = embedding(encoded_title)
# embeddings_title = np.vstack(embeddings_title)
# features = pd.DataFrame(embeddings_title)

# TEST DATA

In [216]:
test_data['title'] = stopwords(lemmatize2(string(clear_text(test_data['title']))))

In [217]:
X_test = test_data['title']

In [218]:
corpus3 = X_test.values.astype('U')

In [219]:
X_test = _vectorizer.transform(corpus3)

In [220]:
test_prediction = model.predict(X_test)

In [221]:
test_data = (test_data
 .join(pd.DataFrame(test_prediction))
 .rename(columns = {0: 'predicted_category_id'})
 )

In [227]:
test_data = test_data[['id', 'predicted_category_id']]

In [228]:
test_data

,id,predicted_category_id
0,1070974,11574
1,450413,11878
2,126857,13299
3,1577569,13061
4,869328,12813
...,...,...
70859,967535,13143
70860,1488636,2674
70861,827510,13324
70862,529244,2599


In [229]:
test_data.to_parquet('result.parquet', engine='pyarrow')

In [226]:
(test_data
 .merge(categories_full, left_on = 'predicted_category_id', right_on = 'id', how = 'left')
 .drop(['id_y', 'parent_id'], axis = 1)
)

,id_x,title_x,short_description,name_value_characteristics,rating,feedback_quantity,predicted_category_id,title_y
0,1070974,браслет натуральный камень,None,None,0.000000,0,11574,Браслеты
1,450413,шампунь сухой окрасить волос личить,None,None,4.333333,6,11878,Шампуни
2,126857,микрофон всенаправленный,"универсальный 3,5 мм микрофон запишет ваш звук",None,3.708333,24,13299,Микрофоны
3,1577569,серьга гвоздик сердце,Серьги гвоздики сердце,None,0.000000,0,13061,Серьги
4,869328,чёрно красный стильный брошь тюльпан акрил бро...,Стильная и яркая брошь ручной работы! Великоле...,None,0.000000,0,12813,Броши
...,...,...,...,...,...,...,...,...
70859,967535,носка мех куница авокадо разноцветный,Пуховые носки с мехомом куницы с авакадо.,None,5.000000,3,13143,Носки и подследники
70860,1488636,эфирный масло сосна кедрмаркет,"Масло сосны повышает защитную функцию кожи, уп...",None,0.000000,0,2674,Эфирные масла
70861,827510,компект футболка шорты,"Отличный комплект. Удобный, комфортный.",None,0.000000,0,13324,Комплекты
70862,529244,купальный костюм,None,Российский размер:40|42|44|46,0.000000,0,2599,Купальники
